# Reinforcement Learning

### Overview

#### What is the defining trait of Reinforment Learning?
- There exists a loop in which the **agent** will receive feedback from the **environment**

#### How do we formulate this feedback loop?
We will define $x_t$ as the state of environment at time $t$.  (i.e. $x_t$ can be defined as a vector which each entry defines the *temperature*, *luminocity*...)

However, an agent may not receive the entire $x_t$.  (A temperature gauge cannot take picture...), we will define $o_t$, which is the observation the agent makes.

$u_t$ is the action agent will take at time $t$.

Our goal is to learn **policy**, $\pi_{\theta}(u_t | o_t)$, which defines which action agent needs to take given an observati

#### More specific definition of policy...

**Behavior Policy**: $\pi_{\theta}(u | o)$ is the policy agent uses to act in the world

**Target Policy**: $\pi_{\theta^t}^t(u | o) $ is the policy agent is learning

### DAGGER:

What is